# Clean Orbis Data

In this notebook, we clean a part of the Orbis dataset containing company addresses.

The notebook is organized in the following fashion:

0. Import libraries and define constants
1. Load parts of Orbis dataset
2. Check the data
3. Clean addresses
4. Translate English names
5. German ZIP codes
6. Fill missing data
7. Clean the result
8. Save processed data

In [ ]:
%load_ext autoreload
%autoreload 2

!pip install modin[all] 

In [ ]:
import os
import ftfy
import pyunpack
import numpy as np
os.environ["MODIN_ENGINE"] = "dask"
import modin.pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline     
sns.set(color_codes=True)

import linkage.model.fill_addresses as fa
import linkage.model.german_zip_codes as gzc

from linkage.model.utils import save_dataframe, read_dataframe
from linkage.model.change_dataframe import replace_german_characters, repair_broken_unicode, replace_other_latin_characters
from linkage.model.clean_addresses import clean_addresses, replace_english_names
from linkage.model.examine_dataframe import contains_all_nan, contains_any_nan, drop_all_nan, count_redundant_spaces
from linkage.model.examine_dataframe import column_contains_nan, drop_subset_nan, print_dataframe_length
from linkage.visualize.plot import plot_histogram
from linkage.visualize.visualize_dataframe import show_nan_counts

In [ ]:
# Two types of data, all or the first part (part01.rar)
# part01 is used for implementation purposes 
# To check if everything is working as it should
TYPE = 'part01'  # 'all' or 'part01'

# 'std' for standardized, 'std_dict_40k' for dictionary cleaning with the 40k most common words
NOTE = 'std_dict_40k'

In [ ]:
# Specify paths to data directories
INTERMEDIATE_DATA_DIR = "../data/intermediate/orbis"
PROCESSED_DATA_DIR = f"../data/processed/orbis/{TYPE}"

# Specifie file names
#ORBIS_FILE = "orbis_german_all_addresses.csv"
ORBIS_FILE = f"orbis_german_all_addresses_unprocessed_{TYPE}_{NOTE}.csv"
ORBIS_PROCESSED_FILE = f"orbis_german_all_addresses_processed_{TYPE}_{NOTE}_small.csv"

# Dataframe's index
ORBIS_INDEX = 'BvD ID number'

# Column names
# Good to specify if the column names would change
COMPANY_CITY, COMPANY_ZIP, COMPANY_STATE = 'City (native)', 'Postcode', 'Region in country'
COMPANY_CITY_INTERNATIONAL, COUNTRY, ISO_CODE = 'City', 'Country', 'Country ISO code',

# Columns to take when reading the dataframe from a file
USEFUL_COLS = [ORBIS_INDEX, COMPANY_CITY, COMPANY_ZIP, COMPANY_STATE,
               COMPANY_CITY_INTERNATIONAL, COUNTRY, ISO_CODE]

# Address columns
COMPANY_ADDR_COLS = [COMPANY_CITY, COMPANY_ZIP, COMPANY_STATE]
COMPANY_ADDR_COLS_ALL = [COMPANY_CITY, COMPANY_ZIP, COMPANY_STATE, COUNTRY, ISO_CODE]

# Split columns to lists if numerical or alpha-numerical
COMPANY_ADDR_COLS_NAMES = [COMPANY_CITY, COMPANY_STATE]
COMPANY_ADDR_COLS_ZIPCODES = [COMPANY_ZIP]

# Columns to use to determine and drop duplicates
DEDUPLICATION_COLS = [ORBIS_INDEX, COMPANY_CITY]

# Labels for plots
PLOT_LABELS_ALL = ['Comp. ZIP code', 'Comp. city', 'Comp. city internat.', 'Country', 'ISO code', 'Comp. state']
PLOT_LABELS = ['Comp. ZIP code', 'Comp. city', 'Comp. state']

## 1. Load parts of Orbis dataset

The Orbis dataset is stored on path:
```python
../data/intermediate/orbis/
```

The data are read into Pandas **DataFrame**.


In [ ]:
# Read previously obtained addresses of German companies
df = read_dataframe(INTERMEDIATE_DATA_DIR, ORBIS_FILE, ORBIS_INDEX, USEFUL_COLS)
print_dataframe_length(df)
df.head()

In [ ]:
#df = df.head(100)

## 2. Check the data

What should be checked:
- Columns' type
- Number of unique rows
- Index
- NaN values
- Broken Unicode 

### Check the dataframe info

First, we check the number of columns and rows.

We print the column names with their data types.

In [ ]:
df.info(verbose=True , show_counts=True)

### Check for uniqueness and index

Then, we look at the uniqueness of values in the individual columns.

Next, we check if the data frame has an index. If there is no index, the execution ends with an exception.

In [ ]:
# Check if the column is unique
for i in df.columns:
  print(f'{i} is unique: {df[i].is_unique}')

# Check the index values
# Results in error if there is no index
df.index

### Check NaN values

Here, we check the missing data.

In [ ]:
show_nan_counts(df, PLOT_LABELS_ALL, ymin=0, ymax=len(df)+1000)

#### All values are NaN

Let's check if some rows are NaN.

In [ ]:
contains_all_nan(df)

#### Deal with all NaN rows

For now, we will drop the rows with only NaN values.

In [ ]:
drop_all_nan(df)

#### Some values are NaN

Let's check if some rows have NaN.

In [ ]:
contains_any_nan(df)

Note: **BvD ID number** is not unique.

### Reset index

For cleaning and consequent updating of the dataframe, we need to reset the index. Otherwise, the update of the main dataframe by cleaned subdataframe will end in error.

In [ ]:
# Reset index
df.reset_index(inplace=True)

### Broken Unicode

It can happen that someone has encoded Unicode with one standard and decoded it with a different one.

As a result, some of the characters may be "broken".

A nice example is ampersand (&) which will decode as &amp.

In [ ]:
# Repair broken unicode
repair_broken_unicode(df, COMPANY_ADDR_COLS_NAMES)
df.head()

### Replace with basic Latin characters

Let's check if the dataframe contains any characters other than basic Latin ones and replace them.

In [ ]:
# Check all rows with other than German alphanumerical characters
df[df[COMPANY_CITY].str.contains('[ÄÖÜßÁÉÓÚ]', regex=True) == True].head()

#### Replace German characters

Replace German characters with umlaut and ß with their basic Latin equivalents.

In [ ]:
# Replace characters with umlaut
replace_german_characters(df, COMPANY_ADDR_COLS_NAMES)

# Check all rows with other than German alphanumerical characters
df[df[COMPANY_CITY].str.contains('[ÄÖÜß]', regex=True) == True].head()

#### Columns contain only German characters

Let's check if the company names contain different than German characters.

In [ ]:
# Replace á to a etc.
replace_other_latin_characters(df, COMPANY_ADDR_COLS_NAMES)

# Check all rows with other than latin alphanumerical characters
df[df[COMPANY_CITY].str.contains('[ÁÉÓÚ]', regex=True) == True].head()

### Deduplication

We check duplicated records and drop them.

We decide duplicates based on the _BvD ID_ and _City (native)_.

In [ ]:
# Show the duplicated records
df[df.duplicated(subset=DEDUPLICATION_COLS, keep=False) == True].sort_values(ORBIS_INDEX).head()

In [ ]:
# Drop duplicates
df.drop_duplicates(subset=DEDUPLICATION_COLS, inplace=True)

# Get the new lenght of the dataframe
print_dataframe_length(df)

# Check again
df[df.duplicated(subset=DEDUPLICATION_COLS, keep=False) == True].sort_values(ORBIS_INDEX).head()

### Check Columns

Let's look at the different values of _Country ISO code_ and _Country_ columns.

In [ ]:
# Print unique values of ISO code
pd.DataFrame(df[ISO_CODE].unique())

In [ ]:
# Plot ISO code values different from 'DE'
df[df[ISO_CODE] != 'DE'][ISO_CODE].value_counts().plot.pie()

In [ ]:
# Plot country values different from 'Germany'
non_german_df = df[df[COUNTRY] != 'Germany']
non_german_df[COUNTRY].value_counts().plot.pie()

In [ ]:
# Drop columns containing other than German addresses
df.drop(df[df[COUNTRY] != 'Germany'].index, inplace=True)

print_dataframe_length(df)

# Check for other countries than Germany
df[df[COUNTRY] != 'Germany']

In [ ]:
# Drop redundant columns 'Country', 'Country ISO code', and 'City' (international)
df.drop(labels=[COUNTRY, ISO_CODE, COMPANY_CITY_INTERNATIONAL], axis=1, inplace=True)

## 3. Clean addresses


Clean non-numerical parts of addresses from non-alphabetical characters, group single consecutive letters, and turn names upper case.

In [ ]:
# Clean company addresses
for column_name in COMPANY_ADDR_COLS_NAMES:

    clean_addresses(df, column_name)
    
df.head()

## 4. Translate English names


Some cities may be named in English.
Translate the English names to their German equivalents.

In [ ]:
# Check for German city name
df[df[COMPANY_CITY].str.contains('MUNICH') == True].head()

In [ ]:
# Translate city native
replace_english_names(df, COMPANY_CITY)

# Check for German city name
df[df[COMPANY_CITY].str.contains('MUNICH') == True].head()

## 5. German ZIP codes

The _German-Zip-Codes.csv_ of German ZIP codes is saved on path
```python
../data/external/german-zip-codes
```

We use German-Zip-Codes to fill the missing values.


In [ ]:
# Initialize class for German-Zip-Codes
german_zipcodes = gzc.GermanZipCodes()

# Read the standardized dataframe of german zip codes
zip_df = german_zipcodes.zip_df
zip_df.head()

In [ ]:
# Check the german-zip-codes dataframe info
zip_df.info(verbose=True , show_counts=True)

#### Replace the mean of ZIP codes

Because we used the mean of the ZIP codes in the previous step, we do not want to use the full ZIP to avoid confusion.

We replace the last 3 numbers of ZIP codes with 'xxx'.

In [ ]:
zip_mean_df = german_zipcodes.zip_mean_df
zip_mean_df.head()

## 6. Fill missing data

Here, we try to fill missing parts of company addresses using other non-missing values of records.

In [ ]:
# Initialize class for cleaning data
fill_address = fa.FillAddress(df, zip_df, zip_mean_df, COMPANY_ZIP, COMPANY_CITY, COMPANY_STATE)

### ZIP codes

Fill missing company ZIP codes.

In [ ]:
# Filter missing or invalid zipcodes and create a new dataframe
missing_zip_mask = (df[COMPANY_ZIP].isna() \
                    | df[COMPANY_ZIP].str.contains('[a-zA-Z]', regex=True) == True) \
                    & df[COMPANY_CITY].notna()

missing_zip_df = df[missing_zip_mask].copy()
missing_zip_df.head()

In [ ]:
# Fill missing zipcode
missing_zip_df = fill_address.fill_missing_zipcode(missing_zip_df)
missing_zip_df.head()

In [ ]:
# Update main dataframe
df.update(missing_zip_df)

#### Check the data

In [ ]:
df[df[COMPANY_ZIP].isna()].head()

### City (native)

Fill missing company cities.

In [ ]:
df[df[COMPANY_CITY].isna()].head()

#### Missing City

In [ ]:
# Filter missing or invalid cities and create a new dataframe
missing_city_mask = (df[COMPANY_CITY].isna() \
                     | (df[COMPANY_CITY].str.contains('[0-9]', regex=True) == True)) \
                     & df[COMPANY_ZIP].notna()

missing_city_df = df[missing_city_mask].copy()

missing_city_df.head()

In [ ]:
# Fill missing city
missing_city_df = fill_address.fill_missing_city(missing_city_df)
missing_city_df.head()

In [ ]:
# Update main dataframe
df.update(missing_city_df)

#### Check the data

In [ ]:
df[df[COMPANY_CITY].isna()]

### Unclear German state

Change Region in country to Bundesland.

In [ ]:
# Filter unclear states (containing a '|') and create a new dataframe
unclear_state_mask = df[COMPANY_STATE].notna() & df[COMPANY_STATE].str.contains('|', regex=False)

unclear_state_df = df[unclear_state_mask].copy()

unclear_state_df.head()

In [ ]:
# Fill unclear state
unclear_state_df = fill_address.fill_unclear_state(unclear_state_df)
unclear_state_df.head()

In [ ]:
# Update main dataframe
df.update(unclear_state_df)

#### Check the data

In [ ]:
df[(df[COMPANY_STATE].notna() & df[COMPANY_STATE].str.contains('|', regex=False))].head()

### German State

Fill missing company states.

In [ ]:
#df[COMPANY_STATE] = df[COMPANY_STATE].astype('category')

In [ ]:
# Filter missing or invalid states and create a new dataframe
missing_state_mask = (df[COMPANY_STATE].isna() 
                      | df[COMPANY_STATE].str.contains('0-9', regex=True)
                     | ~df[COMPANY_STATE].isin(fill_address.bundesland_lst)) \
                      & (df[COMPANY_ZIP].notna() 
                      | df[COMPANY_CITY].notna())

missing_state_df = df[missing_state_mask].copy()
missing_state_df

In [ ]:
# Fill missing state
missing_state_df = fill_address.fill_missing_state(missing_state_df)
missing_state_df

In [ ]:
# Update main dataframe
df.update(missing_state_df)

#### Check the data

In [ ]:
df[df[COMPANY_STATE].isna()].head()

## 7. Clean the result

Here, we again:

- Deduplicate records
- Check NaN values
- Set an index


### Deduplication

After filling missing values, a new duplicated records may appear. Here, we will deal with them.

In [ ]:
# Show the duplicated records
df[df.duplicated(subset=DEDUPLICATION_COLS, keep=False) == True].sort_values(ORBIS_INDEX).head()

In [ ]:
# Drop duplicates
df.drop_duplicates(subset=DEDUPLICATION_COLS, inplace=True)

# Check again
df[df.duplicated(subset=DEDUPLICATION_COLS, keep=False) == True].sort_values(ORBIS_INDEX).head()

### Set an index

In _1. Load parts of Orbis dataset_, we read the data without setting the _BvD ID_ as an index.

We set the index now, so the data saved to file do not contain an additional column with previously used index (line number).

In [ ]:
# Set an index
df.set_index(ORBIS_INDEX, inplace=True)
df.head()

In [ ]:
print_dataframe_length(df)

### Check NaN values

Here, we check the remaining missing data after the cleaning.

We drop the records, which contain only NaN values.

In [ ]:
# Plot num. of NaN values after filling missing values before dropping NaN values
show_nan_counts(df, PLOT_LABELS, ymin=0, ymax=len(df)+1000)

In [ ]:
contains_all_nan(df)

In [ ]:
drop_all_nan(df)

In [ ]:
# Plot num. of NaN values after filling missing values after dropping NaN values
show_nan_counts(df, PLOT_LABELS, ymin=0, ymax=len(df)+1000)

### Check State values

We check counts for each German state.

In [ ]:
# Plot company states after filling
df[COMPANY_STATE].value_counts().plot(kind='bar')

## 8. Save processed data

The processed data is stored in a csv file on path:
```python
../data/processed/orbis
```

In [ ]:
save_dataframe(df, PROCESSED_DATA_DIR, ORBIS_PROCESSED_FILE)